In [19]:
# LOAD & INSPECT DATASET
import pandas as pd

df = pd.read_csv("crop_dataset.csv") # load  dataset

#preview
print(df.shape)
print(df.head)

(69718, 14)
<bound method NDFrame.head of        Unnamed: 0          N         Fe           Mn          S          Mg  \
0               0   90.00000  22.874002  1755.781061   0.478917   99.434611   
1               1   89.84375  27.877571  1225.577971   9.492673  122.533950   
2               2   89.68750  29.953866  1652.877830  11.792660   28.923411   
3               3   89.53125  20.349620   429.346744  13.593830  104.724820   
4               4   89.37500  25.667180  1117.904838   6.952989   90.663107   
...           ...        ...        ...          ...        ...         ...   
69713        3164  105.62500  25.363184    83.085947   4.418164  141.585259   
69714        3165  105.21875  27.265318  1238.304455  12.416972   27.629203   
69715        3166  104.81250  27.457146  1424.860030  13.230450  115.071609   
69716        3167  104.40625  22.176982   260.786061  17.511177   57.416916   
69717        3168  104.00000  26.854282   305.063932   4.932017   60.085623   

         

In [21]:
#Select only the 7 most Relevant Features
#Keep only 7 import columns and relevant features

columns_to_keep = ['N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall', 'label']
df_reduced = df[columns_to_keep]

In [25]:
#handle missing values (if any)
print(df_reduced.isnull().sum()) #check for nulls
df_reduced = df_reduced.dropna() # or use fillna()

N              0
P              0
K              0
temperature    0
humidity       0
ph             0
rainfall       0
label          0
dtype: int64


In [27]:
#Using only 50% of the dataset
#randomly sample 50% of the dataset
df_half = df_reduced.sample(frac=0.5, random_state =42).reset_index(drop=True)

In [30]:
# Normalizing the input features
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(df_half.drop('label', axis=1))
y = df_half['label']


In [32]:
#Encode Labels(Crops)
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)


In [34]:
#Train Test Spit
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)


In [38]:
#Build a Neural Network for (ESP32)
from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Input(shape=(7,)),                      # define input layer properly
    Dense(32, activation='relu'),
    Dense(32, activation='relu'),
    Dense(len(encoder.classes_), activation='softmax')
])


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/20
872/872 ━━━━━━━━━━━━━━━━━━━━ 1s 593us/step - accuracy: 0.4573 - loss: 2.1442 - val_accuracy: 0.9476 - val_loss: 0.3663
Epoch 2/20
872/872 ━━━━━━━━━━━━━━━━━━━━ 0s 507us/step - accuracy: 0.9591 - loss: 0.2793 - val_accuracy: 0.9816 - val_loss: 0.1322
Epoch 3/20
872/872 ━━━━━━━━━━━━━━━━━━━━ 0s 502us/step - accuracy: 0.9817 - loss: 0.1169 - val_accuracy: 0.9857 - val_loss: 0.0776
Epoch 4/20
872/872 ━━━━━━━━━━━━━━━━━━━━ 0s 501us/step - accuracy: 0.9871 - loss: 0.0708 - val_accuracy: 0.9877 - val_loss: 0.0555
Epoch 5/20
872/872 ━━━━━━━━━━━━━━━━━━━━ 0s 503us/step - accuracy: 0.9885 - loss: 0.0529 - val_accuracy: 0.9891 - val_loss: 0.0441
Epoch 6/20
872/872 ━━━━━━━━━━━━━━━━━━━━ 0s 523us/step - accuracy: 0.9902 - loss: 0.0412 - val_accuracy: 0.9898 - val_loss: 0.0356
Epoch 7/20
872/872 ━━━━━━━━━━━━━━━━━━━━ 0s 510us/step - accuracy: 0.9919 - loss: 0.0337 - val_accuracy: 0.9918 - val_loss: 0.0292
Epoch 8/20
872/872 ━━━━━━━━━━━━━━━━━━━━ 0s 538us/step - accuracy: 0.9936 - loss: 0.0281 - 

In [40]:
#Exporting to .tflite
# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save as file
with open("crop_model_ESP.tflite", "wb") as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: /var/folders/s8/w058h8rn5bzf2zx2g30dczpc0000gn/T/tmptr05bocf/assets


INFO:tensorflow:Assets written to: /var/folders/s8/w058h8rn5bzf2zx2g30dczpc0000gn/T/tmptr05bocf/assets


Saved artifact at '/var/folders/s8/w058h8rn5bzf2zx2g30dczpc0000gn/T/tmptr05bocf'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 7), dtype=tf.float32, name='keras_tensor_11')
Output Type:
  TensorSpec(shape=(None, 22), dtype=tf.float32, name=None)
Captures:
  6374773328: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6374772944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6374772752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6374769872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6374773904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6374771600: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1743185824.040877  859421 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1743185824.041195  859421 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-03-28 13:17:04.043397: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/s8/w058h8rn5bzf2zx2g30dczpc0000gn/T/tmptr05bocf
2025-03-28 13:17:04.043859: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-03-28 13:17:04.043864: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/s8/w058h8rn5bzf2zx2g30dczpc0000gn/T/tmptr05bocf
2025-03-28 13:17:04.047097: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-03-28 13:17:04.064046: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /var/folders/s8/w058h8rn5bzf2zx2g30dczpc0000gn/T/tmptr05bocf
2025-03-28 13:17:04.068226: I tensorflow/cc/saved_model/loader.cc:

In [42]:
# Save the TFLite model to a file
with open("crop_model_ESP.tflite", "wb") as f:
    f.write(tflite_model)
